In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime, timedelta
import pytz

credentials = service_account.Credentials.from_service_account_file(
    '../secrets/google_key.json'
)
client = bigquery.Client(credentials=credentials)

project_id = 'mlbhighlights'
dataset_id = 'highlights'
table_id = '2024_pre'

table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

In [3]:

yesterday = datetime.now() - timedelta(1)
yesterday= yesterday.strftime(f'%Y-%m-%d')

today = datetime.now()
today = today.strftime(f'%Y-%m-%d')


def player_search(player, start_date=yesterday, end_date=today):
    query = f"""
        SELECT * 
        FROM {project_id}.{dataset_id}.{table_id}
        WHERE date >= '{start_date}' and date <= '{end_date} 23:59:59' and player_name like concat('%','{player}','%')
    """
    query_job = client.query(query)
    results = query_job.result()

    highlight_list = []
    for row in results:
        date = row[1].astimezone(pytz.timezone('US/Eastern'))
        name = row[2]
        headline = row[4]
        description = row[6]
        url = row[7]
        highlight = [date,name,headline,description,url]
        highlight_list.append(highlight)
    return highlight_list

In [8]:
list = player_search('Holliday','2024-02-01')
for h in list:
    print(h)

[datetime.datetime(2024, 2, 29, 15, 15, 13, 581000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>), 'Jackson Holliday', "Check out Jackson Holliday's RBI triple", "Check out a unique angle of Jackson Holliday's game-tying RBI triple in the bottom of the 6th inning against the Pirates #CreatorCuts", 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-02/29/85fb9b6d-463a784d-294a01e4-csvm-diamondx64-asset_1280x720_59_4000K.mp4']
[datetime.datetime(2024, 2, 29, 16, 4, 11, 475000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>), 'Jackson Holliday', "Radio call of Jackson Holliday's RBI triple", "Listen to the radio call of Jackson Holliday's game-tying RBI triple in the bottom of the 6th inning against the Pirates", 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-02/29/bbd91d72-71d590ca-60ad936a-csvm-diamondx64-asset_1280x720_59_4000K.mp4']
[datetime.datetime(2024, 2, 29, 16, 35, 26, 914000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>), 'Jackson Holliday', "F